In [2]:
import os
os.environ['GRB_LICENSE_FILE'] = r'C:\Users\AKSHAY\gurobi.lic'


In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
n=int(input())

crystals=[]
for _ in range(n):
    a,b,c=map(int,input().split())
    crystals.append((a,b,c))

for row in crystals:
    print(row)

 4
 10 20 100
 40 50 150
 25 35 200
 50 50 300


(10, 20, 100)
(40, 50, 150)
(25, 35, 200)
(50, 50, 300)


In [5]:
m=int(input())

mines=[]
for _ in range(m):
    a,b,c=map(int,input().split())
    mines.append((a,b,c))

for row in mines:
    print(row)

 3
 15 25 50
 70 60 75
 30 200 60


(15, 25, 50)
(70, 60, 75)
(30, 200, 60)


In [6]:


model = gp.Model("model1")

# Define variables for the polygon vertices
x_values = model.addVars(1000, lb=0.0, ub=10000.0, name="x")
y_values = model.addVars(1000, lb=0.0, ub=10000.0, name="y")

# Define binary helper variables for edge orientation
b_vars = model.addVars(1000, vtype=GRB.BINARY, name="b")

M = 10000.0
epsilon = 1e-6

# Ensure each edge is either horizontal or vertical
for i in range(1000):
    prev_i = (i - 1) % 1000
    model.addConstr(x_values[i] - x_values[prev_i] <= M * b_vars[i], name=f"x_upper_{i}")
    model.addConstr(x_values[prev_i] - x_values[i] <= M * b_vars[i], name=f"x_lower_{i}")
    model.addConstr(y_values[i] - y_values[prev_i] <= M * (1 - b_vars[i]), name=f"y_upper_{i}")
    model.addConstr(y_values[prev_i] - y_values[i] <= M * (1 - b_vars[i]), name=f"y_lower_{i}")

model.update()

for i in range(num_vertices - 2):
    for j in range(i + 2, num_vertices - 1):  # Ensure we only compare non-adjacent edges
        if i == 0 and j == num_vertices - 2:  # Ignore first and last edge connection
            continue
        
        # Create auxiliary variables for orientation values
        o1 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o1_{i}_{j}")
        o2 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o2_{i}_{j}")
        o3 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o3_{i}_{j}")
        o4 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o4_{i}_{j}")

        # Compute orientation values using constraints
        model.addConstr(o1 == (y_values[j] - y_values[i]) * (x_values[i+1] - x_values[i]) - (x_values[j] - x_values[i]) * (y_values[i+1] - y_values[i]))
        model.addConstr(o2 == (y_values[j+1] - y_values[i]) * (x_values[i+1] - x_values[i]) - (x_values[j+1] - x_values[i]) * (y_values[i+1] - y_values[i]))
        model.addConstr(o3 == (y_values[i] - y_values[j]) * (x_values[j+1] - x_values[j]) - (x_values[i] - x_values[j]) * (y_values[j+1] - y_values[j]))
        model.addConstr(o4 == (y_values[i+1] - y_values[j]) * (x_values[j+1] - x_values[j]) - (x_values[i+1] - x_values[j]) * (y_values[j+1] - y_values[j]))

        # Binary variables to enforce non-self-intersection
        b1 = model.addVar(vtype=GRB.BINARY, name=f"b1_{i}_{j}")
        b2 = model.addVar(vtype=GRB.BINARY, name=f"b2_{i}_{j}")

        # Big-M constraints to enforce correct intersection conditions
        model.addConstr(o1 <= big_M * b1)
        model.addConstr(o2 >= -big_M * (1 - b1))

        model.addConstr(o3 <= big_M * b2)
        model.addConstr(o4 >= -big_M * (1 - b2))

        # Ensure that at least one condition holds
        model.addConstr(b1 + b2 >= 1, f"no_intersection_{i}_{j}")

# total_score = 0

# # Process crystals
# for idx, (a, b, c) in enumerate(crystals):
#     sum_cross = 0
#     for i in range(1000):
#         prev_i = (i - 1) % 1000
#         edge_vertical = 1 - b_vars[i]

#         x_gt = model.addVar(vtype=GRB.BINARY, name=f"x_gt_c_{i}_{idx}")
#         model.addConstr(x_values[prev_i] >= a + epsilon - M * (1 - x_gt), name=f"x_gt_constr1_{i}_{idx}")
#         model.addConstr(x_values[prev_i] <= a + M * x_gt, name=f"x_gt_constr2_{i}_{idx}")

#         y_prev = y_values[prev_i]
#         y_curr = y_values[i]
#         y_low = model.addVar(vtype=GRB.BINARY, name=f"y_low_c_{i}_{idx}")
#         model.addConstr(y_prev <= b + M * (1 - y_low), name=f"y_low_constr1_{i}_{idx}")
#         model.addConstr(y_curr >= b - M * (1 - y_low), name=f"y_low_constr2_{i}_{idx}")

#         y_high = model.addVar(vtype=GRB.BINARY, name=f"y_high_c_{i}_{idx}")
#         model.addConstr(y_prev >= b - M * (1 - y_high), name=f"y_high_constr1_{i}_{idx}")
#         model.addConstr(y_curr <= b + M * (1 - y_high), name=f"y_high_constr2_{i}_{idx}")

#         y_cross = model.addVar(vtype=GRB.BINARY, name=f"y_cross_c_{i}_{idx}")
#         model.addConstr(y_cross >= y_low + y_high - 1, name=f"y_cross_constr1_{i}_{idx}")
#         model.addConstr(y_cross <= y_low, name=f"y_cross_constr2_{i}_{idx}")
#         model.addConstr(y_cross <= y_high, name=f"y_cross_constr3_{i}_{idx}")

#         cross = model.addVar(vtype=GRB.BINARY, name=f"cross_c_{i}_{idx}")
#         model.addConstr(cross <= x_gt, name=f"cross_constr1_{i}_{idx}")
#         model.addConstr(cross <= y_cross, name=f"cross_constr2_{i}_{idx}")
#         model.addConstr(cross <= edge_vertical, name=f"cross_constr3_{i}_{idx}")
#         model.addConstr(cross >= x_gt + y_cross + edge_vertical - 2, name=f"cross_constr4_{i}_{idx}")

#         sum_cross += cross

#     k = model.addVar(vtype=GRB.INTEGER, name=f"k_c_{idx}")
#     inside = model.addVar(vtype=GRB.BINARY, name=f"inside_c_{idx}")
#     model.addConstr(sum_cross == 2 * k + inside, name=f"inside_constr_c_{idx}")
#     total_score += c * inside

# # Process mines
# for idx, (a, b, c) in enumerate(mines):
#     sum_cross = 0
#     for i in range(1000):
#         prev_i = (i - 1) % 1000
#         edge_vertical = 1 - b_vars[i]

#         x_gt = model.addVar(vtype=GRB.BINARY, name=f"x_gt_m_{i}_{idx}")
#         model.addConstr(x_values[prev_i] >= a + epsilon - M * (1 - x_gt), name=f"x_gt_constr1_{i}_{idx}_m")
#         model.addConstr(x_values[prev_i] <= a + M * x_gt, name=f"x_gt_constr2_{i}_{idx}_m")

#         y_prev = y_values[prev_i]
#         y_curr = y_values[i]
#         y_low = model.addVar(vtype=GRB.BINARY, name=f"y_low_m_{i}_{idx}")
#         model.addConstr(y_prev <= b + M * (1 - y_low), name=f"y_low_constr1_{i}_{idx}_m")
#         model.addConstr(y_curr >= b - M * (1 - y_low), name=f"y_low_constr2_{i}_{idx}_m")

#         y_high = model.addVar(vtype=GRB.BINARY, name=f"y_high_m_{i}_{idx}")
#         model.addConstr(y_prev >= b - M * (1 - y_high), name=f"y_high_constr1_{i}_{idx}_m")
#         model.addConstr(y_curr <= b + M * (1 - y_high), name=f"y_high_constr2_{i}_{idx}_m")

#         y_cross = model.addVar(vtype=GRB.BINARY, name=f"y_cross_m_{i}_{idx}")
#         model.addConstr(y_cross >= y_low + y_high - 1, name=f"y_cross_constr1_{i}_{idx}_m")
#         model.addConstr(y_cross <= y_low, name=f"y_cross_constr2_{i}_{idx}_m")
#         model.addConstr(y_cross <= y_high, name=f"y_cross_constr3_{i}_{idx}_m")

#         cross = model.addVar(vtype=GRB.BINARY, name=f"cross_m_{i}_{idx}")
#         model.addConstr(cross <= x_gt, name=f"cross_constr1_{i}_{idx}_m")
#         model.addConstr(cross <= y_cross, name=f"cross_constr2_{i}_{idx}_m")
#         model.addConstr(cross <= edge_vertical, name=f"cross_constr3_{i}_{idx}_m")
#         model.addConstr(cross >= x_gt + y_cross + edge_vertical - 2, name=f"cross_constr4_{i}_{idx}_m")

#         sum_cross += cross

#     k = model.addVar(vtype=GRB.INTEGER, name=f"k_m_{idx}")
#     inside = model.addVar(vtype=GRB.BINARY, name=f"inside_m_{idx}")
#     model.addConstr(sum_cross == 2 * k + inside, name=f"inside_constr_m_{idx}")
#     total_score -= c * inside

# model.setObjective(total_score, GRB.MAXIMIZE)
# model.optimize()

# # Output the optimal total score
# print(f"Optimal Total Score: {model.objVal}")

Set parameter Username
Set parameter LicenseID to value 2616846
Academic license - for non-commercial use only - expires 2026-01-31


In [7]:
# Binary variables to check if a point is inside the polygon
inside = model.addVars(n + m, vtype=GRB.BINARY, name="inside")

# Binary variables to count crossings for ray tracing
crossing_vars = model.addVars(n + m, 1000, vtype=GRB.BINARY, name="crossing")

M = 10000  # Large constant for big-M constraints

# Apply ray tracing for each crystal and mine
for idx, (a, b, c) in enumerate(crystals + mines):
    crossing_sum = gp.LinExpr(0)  # To count total crossings

    for i in range(1000):
        j = (i + 1) % 1000  # Next vertex in the polygon

        # Edge endpoints
        x1, y1 = x_values[i], y_values[i]
        x2, y2 = x_values[j], y_values[j]

        # Define binary variable crossing_vars[idx, i] = 1 if ray crosses edge (i, j)
        model.addConstr(crossing_vars[idx, i] <= (b - y1) * (b - y2) * (-1), name=f"crossing_cond1_{idx}_{i}")
        model.addConstr(crossing_vars[idx, i] <= (a - x1), name=f"crossing_cond2_{idx}_{i}")
        model.addConstr(crossing_vars[idx, i] >= (b - y1) * (b - y2) * (-1) + (a - x1) - 1, name=f"crossing_cond3_{idx}_{i}")

        # Sum up crossings
        crossing_sum += crossing_vars[idx, i]

    # Ensure `inside[idx]` captures the odd/even nature of crossings
    aux = model.addVar(vtype=GRB.INTEGER, name=f"aux_{idx}")
    model.addConstr(crossing_sum == 2 * aux + inside[idx])

# Compute total score
total_score = gp.LinExpr(0)

for i in range(n):
    total_score += inside[i] * crystals[i][2]

for i in range(m):
    total_score -= inside[n + i] * mines[i][2]




In [33]:
# Set objective
model.setParam(GRB.Param.IterationLimit, 10)  # Stop after 10,000 iterations

model.setObjective(total_score, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Output the optimal score
if model.status == GRB.INTERRUPTED:
    print(f"Optimal Score: {model.objVal}")
else:
    print("No optimal solution found.")
    for i in range(1000):
        print(f"Vertex {i}: ({x_values[i].X:.1f}, {y_values[i].X:.1f})")


Set parameter IterationLimit to value 10
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
IterationLimit  10
TimeLimit  60

Optimize a model with 11007 rows, 2006014 columns and 33014 nonzeros
Model fingerprint: 0x3edad529
Model has 2004012 quadratic constraints
Variable types: 2000 continuous, 2004014 integer (2004007 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+04]
  Objective range  [5e+01, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+01, 1e+04]
  QRHS range       [4e+02, 4e+04]
Presolve removed 6000 rows and 1000994 columns (presolve time = 7s)...
Presolve removed 6000 rows and 1000994 columns (presolve time = 13s)...
Presolve removed 7000 rows and 1000994 columns

AttributeError: Unable to retrieve attribute 'X'

In [32]:

   
    for i in range(1000):
        print(f"Vertex {i}: ({x_values[i].X:.1f}, {y_values[i].X:.1f})")


AttributeError: Unable to retrieve attribute 'X'